# 最简单版本
使用 LR 模型进行预测，且无脑使用所有特征且不做转换

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from scipy.stats import skew
from scipy.stats import pearsonr

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [16]:
print(train.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [55]:
# 数据处理
# 1. 删除所有有 Nan 的列
# 2. 只选取类型为 number 的列
def process_data(df: pd.DataFrame):
    # axis=1: remove column
    df = df.dropna(axis=1, how='any')
    df = df.select_dtypes(include='number')
    return df
# 训练数据, 排除 SalePrice 列
train_data = train.loc[:, train.columns != 'SalePrice']
train_label = train.SalePrice
# 生成初识训练、测试数据集
train_processed = process_data(train_data)
test_processed = process_data(test)
# 选取两组数据集的列交接
columns = set(train_processed.columns)
columns = columns.intersection(set(test_processed.columns))
columns = list(columns)
# 生成最终用的训练、测试数据集
train_processed = train_processed[columns]
test_processed = test_processed[columns]

In [56]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(train_processed, train_label, test_size=0.3, random_state=10)
train_X.head()

,MoSold,Id,2ndFlrSF,ScreenPorch,HalfBath,YrSold,GrLivArea,EnclosedPorch,MSSubClass,MiscVal,...,OverallQual,LowQualFinSF,1stFlrSF,FullBath,BedroomAbvGr,YearBuilt,Fireplaces,KitchenAbvGr,WoodDeckSF,3SsnPorch
912,7,913,0,120,0,2006,1015,112,30,620,...,5,0,1015,1,3,1925,0,1,0,0
373,11,374,0,0,0,2009,1319,0,20,0,...,5,0,1319,1,3,1953,0,1,66,0
20,11,21,1218,0,1,2006,2376,0,60,0,...,8,0,1158,3,4,2005,1,1,240,0
800,5,801,1072,0,1,2008,1688,185,60,400,...,6,0,616,2,4,1997,0,1,403,0
152,6,153,896,0,1,2006,1993,0,60,0,...,6,0,1097,2,4,1971,1,1,0,0


In [58]:
from sklearn import linear_model as lm

# 训练 lr 模型
lr = lm.LogisticRegression()
lr.fit(train_X, train_Y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [59]:
from sklearn.metrics import accuracy_score
predict_Y = lr.predict(test_X)
accuracy_score(predict_Y, test_Y)

0.0136986301369863

predict test dataset, and submit

In [60]:
text_predict_Y = lr.predict(test_processed)

In [61]:
print(len(text_predict_Y), len(test))

1459 1459


准备提交数据

In [65]:
!mkdir output

In [66]:
submission_data = pd.DataFrame({'Id': test.Id, 'SalePrice': text_predict_Y})
# print(my_submission)
submission_data.to_csv('output/submission.csv', index=False)

In [1]:
!ls output
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f output/submission.csv -m "lr"

submission.csv
  0%|                                               | 0.00/17.1k [00:00<?, ?B/s]^C
  0%|                                               | 0.00/17.1k [00:56<?, ?B/s]
User cancelled operation


score: 0.37272